In [ ]:
!pip install -q torch_snippets


import torch
from torch_snippets import *
device = "cuda" if torch.cuda.is_available() else "cpu"
from torchvision.utils import make_grid, save_image
from PIL import Image
import torchvision
from torchvision import transforms
import torchvision.utils as vutils
from glob import glob
import cv2
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 102.5 MB/s eta 0:00:00


https://www.kaggle.com/datasets/ashwingupta3012/male-and-female-faces-dataset

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d ashwingupta3012/male-and-female-faces-dataset

Dataset URL: https://www.kaggle.com/datasets/ashwingupta3012/male-and-female-faces-dataset
License(s): CC0-1.0
 99% 1.62G/1.63G [00:08<00:00, 214MB/s]
100% 1.63G/1.63G [00:08<00:00, 197MB/s]


In [ ]:
# Extract the dataset
!unzip male-and-female-faces-dataset.zip -d Male-and-Female-face-dataset

Streaming output truncated to the last 5000 lines.
  inflating: Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (1367).jpg  
  inflating: Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (1368).jpg  
  inflating: Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (1369).jpg  
  inflating: Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (137).jpg  
  inflating: Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (1370).jpg  
  inflating: Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (1371).jpg  
  inflating: Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (1372).jpg  
  inflating: Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (1373).jpg  
  inflating: Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (1374).jpg  
  inflating: Male-and-Female-face-dataset/Male and Female f

In [ ]:
female_path = '/content/Male-and-Female-face-dataset/Male and Female face dataset/Female Faces'
male_path = '/content/Male-and-Female-face-dataset/Male and Female face dataset/Male Faces'
female_images = glob(f'{female_path}/*.jpg') + glob(f'{female_path}/*.png') + glob(f'{female_path}/*.jpeg')
male_images =  glob(f'{male_path}/*.jpg') + glob(f'{male_path}/*.png') + glob(f'{male_path}/*.jpeg')

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
from glob import glob
female_images = glob("/content/Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/*")

print(f"Found {len(female_images)} female images.")

Found 2698 female images.


In [ ]:
female_images[0]

'/content/Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (1588).jpg'

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image

c = 0

female_dir = "cropped_faces_female"
male_dir = "cropped_faces_male"

os.makedirs(female_dir, exist_ok=True)
os.makedirs(male_dir, exist_ok=True)

def crop_faces(image_list, output_dir):
    global c
    for i, image_path in enumerate(image_list):
        try:
            img = Image.open(image_path)
            img = np.array(img)
            if len(img.shape) == 3 and img.shape[2] == 3:
                gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            else:
                c += 1
                continue

            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            if len(faces) == 0:
                print(f"No face found in {image_path}")
                continue  # Skip if no faces are detected

            for (x, y, w, h) in faces:
                img2 = img[y:y + h, x:x + w]
                save_path = os.path.join(output_dir, f"{i}.jpg")
                cv2.imwrite(save_path, cv2.cvtColor(img2, cv2.COLOR_RGB2BGR))
                print(f"Saved cropped face to {save_path}")

        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            c += 1
            continue

print("Processing female images...")
crop_faces(female_images, female_dir)

print("Processing male images...")
crop_faces(male_images, male_dir)

print(f"Completed with {c} images skipped.")


Streaming output truncated to the last 5000 lines.
Saved cropped face to cropped_faces_female/384.jpg
No face found in /content/Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (1327).jpg
Saved cropped face to cropped_faces_female/386.jpg
No face found in /content/Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (1145).jpg
Saved cropped face to cropped_faces_female/388.jpg
Saved cropped face to cropped_faces_female/389.jpg
Saved cropped face to cropped_faces_female/389.jpg
Saved cropped face to cropped_faces_female/390.jpg
Saved cropped face to cropped_faces_female/391.jpg
No face found in /content/Male-and-Female-face-dataset/Male and Female face dataset/Female Faces/0 (2042).jpg
Saved cropped face to cropped_faces_female/393.jpg
Saved cropped face to cropped_faces_female/394.jpg
Saved cropped face to cropped_faces_female/395.jpg
Saved cropped face to cropped_faces_female/396.jpg
No face found in /content/Male-and-Female-face-dataset/Mal

In [ ]:
transform=transforms.Compose([
                               transforms.Resize(64),
                               transforms.CenterCrop(64),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])

In [ ]:
class Faces(Dataset):
    def __init__(self, folders):
        super().__init__()
        self.folderfemale = folders[0]
        self.foldermale = folders[1]
        self.images=sorted(glob(f'{self.foldermale}/*.jpg'))+sorted(glob(f'{self.folderfemale}/*.jpg'))
    def __len__(self):
        return len(self.images)
    def __getitem__(self, ix):
        image_path = self.images[ix]
        image = Image.open(image_path)
        image = transform(image)
        gender = np.where('female' in str(image_path),1,0)
        return image, torch.tensor(gender).long()

In [ ]:
ds = Faces(folders=['cropped_faces_female','cropped_faces_male'])
dataloader = DataLoader(ds, batch_size=64, shuffle=True)

In [ ]:
len(dataloader)

61

In [ ]:
len(ds)

3886

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, emb_size=32):
        super(Discriminator, self).__init__()
        self.emb_size = emb_size
        self.label_embeddings = nn.Embedding(2, self.emb_size)
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64 * 8, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten()
        )
        self.model2 = nn.Sequential(
            nn.Linear(288, 100),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(100, 1),
            nn.Sigmoid()
        )
        self.apply(weights_init)

    def forward(self, input, labels=None):  # Make labels optional
        x = self.model(input)
        if labels is None:
            labels = torch.zeros(input.size(0), dtype=torch.long, device=input.device)  # Dummy labels for summary
        y = self.label_embeddings(labels)
        input = torch.cat([x, y], 1)
        final_output = self.model2(input)
        return final_output


In [ ]:
from torchsummary import summary
discriminator = Discriminator().to(device)
summary(discriminator, (3, 64, 64))  # Only pass the image size, not tensors


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           3,072
         LeakyReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3          [-1, 128, 16, 16]         131,072
       BatchNorm2d-4          [-1, 128, 16, 16]             256
         LeakyReLU-5          [-1, 128, 16, 16]               0
            Conv2d-6            [-1, 256, 8, 8]         524,288
       BatchNorm2d-7            [-1, 256, 8, 8]             512
         LeakyReLU-8            [-1, 256, 8, 8]               0
            Conv2d-9            [-1, 512, 4, 4]       2,097,152
      BatchNorm2d-10            [-1, 512, 4, 4]           1,024
        LeakyReLU-11            [-1, 512, 4, 4]               0
           Conv2d-12             [-1, 64, 2, 2]         524,288
      BatchNorm2d-13             [-1, 64, 2, 2]             128
        LeakyReLU-14             [-1, 6

In [ ]:
class Generator(nn.Module):
    def __init__(self, emb_size=32):
        super(Generator, self).__init__()
        self.emb_size = emb_size
        self.label_embeddings = nn.Embedding(2, self.emb_size)
        self.model = nn.Sequential(
            nn.ConvTranspose2d(100 + self.emb_size, 64 * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(64 * 2, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
        self.apply(weights_init)

    def forward(self, input_noise, labels=None):
        if labels is None:
            labels = torch.zeros(input_noise.size(0), dtype=torch.long, device=input_noise.device)
        label_embeddings = self.label_embeddings(labels).view(len(labels), self.emb_size, 1, 1)
        input = torch.cat([input_noise, label_embeddings], 1)
        return self.model(input)


In [ ]:
generator = Generator().to(device)
summary(generator, (100, 1, 1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Embedding-1                   [-1, 32]              64
   ConvTranspose2d-2            [-1, 512, 4, 4]       1,081,344
       BatchNorm2d-3            [-1, 512, 4, 4]           1,024
              ReLU-4            [-1, 512, 4, 4]               0
   ConvTranspose2d-5            [-1, 256, 8, 8]       2,097,152
       BatchNorm2d-6            [-1, 256, 8, 8]             512
              ReLU-7            [-1, 256, 8, 8]               0
   ConvTranspose2d-8          [-1, 128, 16, 16]         524,288
       BatchNorm2d-9          [-1, 128, 16, 16]             256
             ReLU-10          [-1, 128, 16, 16]               0
  ConvTranspose2d-11           [-1, 64, 32, 32]         131,072
      BatchNorm2d-12           [-1, 64, 32, 32]             128
             ReLU-13           [-1, 64, 32, 32]               0
  ConvTranspose2d-14            [-1, 3,

In [ ]:
def noise(size):
    n = torch.randn(size, 100, 1, 1, device=device)
    return n.to(device)

In [ ]:
def discriminator_train_step(real_data, real_labels, fake_data, fake_labels):
    d_optimizer.zero_grad()
    prediction_real = discriminator(real_data, real_labels)
    error_real = loss(prediction_real, torch.ones(len(real_data), 1).to(device))
    prediction_fake = discriminator(fake_data, fake_labels)
    error_fake = loss(prediction_fake, torch.zeros(len(fake_data), 1).to(device))
    total_loss = error_real + error_fake
    total_loss.backward()
    d_optimizer.step()
    return error_real + error_fake

In [ ]:
def generator_train_step(fake_data, fake_labels):
    g_optimizer.zero_grad()
    prediction = discriminator(fake_data, fake_labels)
    error = loss(prediction, torch.ones(len(fake_data), 1).to(device))
    error.backward()
    g_optimizer.step()
    return error

In [ ]:
discriminator = Discriminator().to(device)
generator = Generator().to(device)
loss = nn.BCELoss()
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
fixed_noise = torch.randn(64, 100, 1, 1, device=device)
fixed_fake_labels = torch.LongTensor([0]*(len(fixed_noise)//2) + [1]*(len(fixed_noise)//2)).to(device)
loss = nn.BCELoss()
n_epochs = 100  # asli 250
img_list = []

In [ ]:
def save_generated_samples(epoch, sample_images, output_dir="face_generated_images"):
    os.makedirs(output_dir, exist_ok=True)
    fname = os.path.join(output_dir, f"generated-images-{epoch}.png")
    save_image(sample_images[-64:], fname, nrow=8, normalize=True)

In [ ]:
from torch_snippets.torch_loader import Report
log = Report(n_epochs)
for epoch in range(n_epochs):
    N = len(dataloader)
    for bx, (images, labels) in enumerate(dataloader):
        real_data, real_labels = images.to(device), labels.to(device)
        fake_labels = torch.LongTensor(np.random.randint(0, 2, len(real_data))).to(device)
        fake_data = generator(noise(len(real_data)), fake_labels)
        fake_data = fake_data.detach()
        d_loss = discriminator_train_step(real_data, real_labels, fake_data, fake_labels)
        fake_labels = torch.LongTensor(np.random.randint(0, 2, len(real_data))).to(device)
        fake_data = generator(noise(len(real_data)), fake_labels).to(device)
        g_loss = generator_train_step(fake_data, fake_labels)
        pos = epoch + (1+bx)/N
        log.record(pos, d_loss=d_loss.detach(), g_loss=g_loss.detach(), end='\r')
    log.report_avgs(epoch+1)
    with torch.no_grad():
        fake = generator(fixed_noise, fixed_fake_labels).detach().cpu()
        save_generated_samples(epoch, fake)
        imgs = vutils.make_grid(fake, padding=2, normalize=True).permute(1,2,0)
        img_list.append(imgs)
        show(imgs, sz=10)